In [39]:
import tensorflow as tf
import tensorflow as tf
import numpy as np

from turtle import up
import pandas as pd
import scipy
from scipy.stats import iqr


def windowed_dataset(dataset: tf.data.Dataset, window_size=5, shift=1, stride=1):
    if window_size == None or window_size <= 0:
        return dataset

    def sub_to_batch(t1, t2=None):
        t1_batches = t1.batch(window_size, drop_remainder=True)

        if t2 is not None:
            t2_batches = t2.batch(window_size, drop_remainder=True)
            return tf.data.Dataset.zip(t1_batches, t2_batches)
        else:
            return t1_batches

    return dataset.window(window_size, shift=shift, stride=stride, drop_remainder=True).flat_map(sub_to_batch)
    return windows  #.batch(window_size, drop_remainder=drop_remainder)


def window_batch_dataset(df: pd.DataFrame, window_size=4, batch_size=None):

    if not batch_size == None and batch_size > 0:
        dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), window_size).batch(batch_size=batch_size, drop_remainder=True)
    else:
        dataset = windowed_dataset(tf.data.Dataset.from_tensor_slices(df.values), window_size)
    return dataset


def convert_to_numpy(ds: tf.data.Dataset):
    # 将 dataset 转换为 NumPy 数组
    numpy_array = None
    for data in ds.as_numpy_iterator():
        # 确保数据是 NumPy 数组
        data_np = np.array(data)

        # 如果 numpy_array 还没有初始化，初始化它
        if numpy_array is None:
            numpy_array = data_np
        else:
            # # 确保数据的形状与 numpy_array 相同
            # if data_np.shape[1:] != numpy_array.shape[1:]:
            #     raise ValueError("All data items must have the same shape except for the first dimension.")

            # 垂直堆叠数据
            numpy_array = np.vstack((numpy_array, data_np))

    return numpy_array


def get_shape(*args):
    shapes = []
    for arg in args:
        shapes.append(arg.shape if arg is not None else None)
    return shapes

In [40]:
rows = 10
columns = ['a', 'b']  #['a', 'b', 'c']
cols = len(columns)
data_x = np.array([[i for c in np.arange(1, cols + 1)] for i in np.arange(1, rows + 1)])

xdf = pd.DataFrame(data_x, columns=columns)

data_y = np.array([np.random.randint(0, high=2, size=None, dtype=int) for i in np.arange(1, rows + 1)])

ydf = pd.DataFrame(data_y, columns=['label'])
print(data_y)


[0 0 0 0 0 0 0 1 0 1]


In [46]:
win_size = 5
batch_size = 5
dsx = window_batch_dataset(xdf, win_size, batch_size)

dsy = window_batch_dataset(ydf, win_size, batch_size)

for _, _y in zip(convert_to_numpy(dsx), convert_to_numpy(dsy)):

    print(get_shape(_, _y))
    print(_, '-----', '---\r\n', _y)

[(5, 2), (5, 1)]
[[1 1]
 [2 2]
 [3 3]
 [4 4]
 [5 5]] ----- ---
 [[0]
 [0]
 [0]
 [0]
 [0]]
[(5, 2), (5, 1)]
[[2 2]
 [3 3]
 [4 4]
 [5 5]
 [6 6]] ----- ---
 [[0]
 [0]
 [0]
 [0]
 [0]]
[(5, 2), (5, 1)]
[[3 3]
 [4 4]
 [5 5]
 [6 6]
 [7 7]] ----- ---
 [[0]
 [0]
 [0]
 [0]
 [0]]
[(5, 2), (5, 1)]
[[4 4]
 [5 5]
 [6 6]
 [7 7]
 [8 8]] ----- ---
 [[0]
 [0]
 [0]
 [0]
 [1]]
[(5, 2), (5, 1)]
[[5 5]
 [6 6]
 [7 7]
 [8 8]
 [9 9]] ----- ---
 [[0]
 [0]
 [0]
 [1]
 [0]]
